## <img src="https://lh3.googleusercontent.com/mUTbNK32c_DTSNrhqETT5aQJYFKok2HB1G2nk2MZHvG5bSs0v_lmDm_ArW7rgd6SDGHXo0Ak2uFFU96X6Xd0GQ=w160-h128" width="45" valign="top" alt="BigQuery"> Generating Business Insights for our Customer Review data using GenAI


### License

In [ ]:
##################################################################################
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     https://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
###################################################################################

### Notebook Overview

- This notebook will use GenAI to generate insights from our Customer Review data.  The customer reviews have had their sentiment determined and the theme(s) ("Good value", "Long Wait Time") extracted. This data will be aggregated for the past 7 days and then passed to the LLM for analysis.  The LLM can provide insights and ideas on how to improve our business.  We are using the LLM like a consultant as well as to generate fresh ideas.

- Notebook Logic:
    1. Create a row, per city, in the customer_review_gen_ai_insight table.  
        - For each city:
            - The sentiment and themes will be aggregated for the past 7 days by calling BigQuery.
            - BigQuery will return the results as JSON.
            - Our LLM prompt will be created:
                - The prompt will ask for insights to be created.
                - The prompt will ask for unconventional ideas.
                - The prompt will ask the LLM to provide an explaination of its thought process.
                - The prompt will provide example JSON results so the LLM knows the output format to generate the results.
                - The prompt will include the results from BigQuery as the context.
                - The prompt will contain an explaination of the BigQuery data.
            - A row will be inserted into the customer_review_gen_ai_insight table.

    2. An update statement will be run on the customer_review_gen_ai_insight table which will call GENERATE_TEXT (gemini-pro) to execute the LLM prompt.  
        - We save our prompt in the table so we can later inspect it or reprocess the prompt.

    3. The results of the GENERATE_TEXT are JSON so the inner JSON is then parsed into the generated_insight_text and generated_insight_json fields.

    4. The data is then displayed.

## Initialize Python

In [ ]:
# The four cities we want to process.
city_names=["New York City", "London", "Tokyo", "San Francisco"]
city_ids=[1,2,3,4]
number_of_coffee_trucks = "4"

In [ ]:
from google.cloud import bigquery
client = bigquery.Client()

## Create the GenAI Insights table

In [ ]:
%%bigquery
CREATE TABLE IF NOT EXISTS `${project_id}.${bigquery_data_beans_curated_dataset}.customer_review_gen_ai_insight`
--CREATE OR REPLACE TABLE `${project_id}.${bigquery_data_beans_curated_dataset}.customer_review_gen_ai_insight` -- only use to replace the entire table
(
    customer_review_gen_ai_insight_id     INTEGER   NOT NULL OPTIONS(description="Primary key."),
    customer_review_gen_ai_insight_type  STRING    NOT NULL OPTIONS(description="The type of insight: WEATHER, EVENT, CAMPAIGN, SOCIAL, WAIT-TIME, SALES, VIDEO-AI"),
    insight_datetime                     TIMESTAMP NOT NULL OPTIONS(description="The datetime of the GenAI insight."),
    applies_to_entity_type               STRING    NOT NULL OPTIONS(description="The type of entity to which the insight applies: Company, City, Truck, Customer"),
    applies_to_entity_id                 INTEGER   NOT NULL OPTIONS(description="The id (primary key) of entity to which the insight applies."),
    applies_to_entity_name               STRING    NOT NULL OPTIONS(description="The name to which the insight applies."),
    llm_prompt                           STRING    NOT NULL OPTIONS(description="The LLM prompt."),
    ml_generate_json_result              JSON               OPTIONS(description="The raw JSON output of the LLM."),
    generated_insight_text               STRING             OPTIONS(description="The generated insight in text"),
    generated_insight_json               JSON               OPTIONS(description="The generated insight in JSON")
)
CLUSTER BY customer_review_gen_ai_insight_id;

In [ ]:
%%bigquery

-- Remove the current days items (so you can re-run)
DELETE
  FROM `${project_id}.${bigquery_data_beans_curated_dataset}.customer_review_gen_ai_insight`
 WHERE CAST(insight_datetime AS DATE) = CURRENT_DATE()
   AND customer_review_gen_ai_insight_type = 'SOCIAL-CREATIVE';

## Supporting Functions

In [ ]:
# Runs a query against BigQuery and waits for it to complete
def RunQuery(sql):
  import time

  if (sql.startswith("SELECT") or sql.startswith("WITH")):
      df_result = client.query(sql).to_dataframe()
      return df_result
  else:
    job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    while query_job.state != "DONE":
      time.sleep(2)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    if query_job.error_result == None:
      return True
    else:
      return False

In [ ]:
# Since our Primary keys are INTs we get the next available value
def GetNextPrimaryKey(fully_qualified_table_name, field_name):
  sql = f"""
  SELECT IFNULL(MAX({field_name}),0) AS result
    FROM `{fully_qualified_table_name}`
  """
  # print(sql)
  df_result = client.query(sql).to_dataframe()
  # display(df_result)
  return df_result['result'].iloc[0] + 1

## Create the LLM prompt for each city and insert into or GenAI Insights table.

In [ ]:
# Get the next primary key
customer_review_gen_ai_insight_id = GetNextPrimaryKey("${project_id}.${bigquery_data_beans_curated_dataset}.customer_review_gen_ai_insight","customer_review_gen_ai_insight_id")

# Loop for each city
for city_index in range(0, 4):
  print(f"Processing city: {city_ids[city_index]}")

  ##############################################################################
  # Get items to process
  ##############################################################################
  sql=f"""WITH max_data AS
          (
            SELECT MAX(review_datetime) AS max_review_datetime
              FROM `${project_id}.${bigquery_data_beans_curated_dataset}.customer_review` AS customer_review
                  INNER JOIN `${project_id}.${bigquery_data_beans_curated_dataset}.city` AS city
                          ON city.city_id = {city_ids[city_index]}
          )
          , theme_data AS
          (
            SELECT customer_review.location_id,
                  location_history.address,
                  CAST(JSON_VALUE(themes,'$.theme') AS STRING) AS review_theme,
                  customer_review.review_sentiment,
                  COUNT(*) AS theme_count
              FROM `${project_id}.${bigquery_data_beans_curated_dataset}.customer_review` AS customer_review
                  CROSS JOIN UNNEST(JSON_QUERY_ARRAY(llm_detected_theme,'$')) AS themes
                  CROSS JOIN max_data
                  INNER JOIN `${project_id}.${bigquery_data_beans_curated_dataset}.city` AS city
                          ON city.city_id = {city_ids[city_index]}
                  INNER JOIN `${project_id}.${bigquery_data_beans_curated_dataset}.location_history` AS location_history
                          ON customer_review.location_id = location_history.location_id
                          AND customer_review.review_datetime BETWEEN location_history.start_datetime AND location_history.stop_datetime
            WHERE customer_review.review_datetime BETWEEN TIMESTAMP_SUB(max_data.max_review_datetime, INTERVAL 7 DAY)
                                                      AND max_data.max_review_datetime
            GROUP BY 1,2,3,4
          )
          SELECT TO_JSON(theme_data) AS customer_review_json
            FROM theme_data
          ORDER BY location_id;
          """

  # Run the SQL to get the customer reviews
  print(f"Gathering data for: {city_names[city_index]}")
  customer_reviews_df = RunQuery(sql)

  # Loop through the dataframe (we want a JSON string for our prompt)
  customer_reviews_json = ""
  for index, row in customer_reviews_df.iterrows():
    customer_reviews_json = customer_reviews_json + row['customer_review_json'] + "\n"

  ##############################################################################
  # Create the LLM prompt
  ##############################################################################

  # Example return JSON (without the python f"" formatting)
  json_example_data='[ { "location_id" : 1, "recommended_action": "Move the truck to the following address.", "address" : "9031 Spring St. New York, NY 10033", "explanation" : "This location has lots of seating." }, { "location_id" : 2, "recommended_action": "Train the employees.", "address" : "1 Queen Street London E62 8YT", "explanation" : "The lack of service means we need additional training." } ]'

  # Use this prompt for Creative responses.
  CREATIVE_llm_prompt=f"""You run a fleet of 4 coffee trucks in {city_names[city_index]}.
Below is a list of customer reviews.
Based upon the customer reviews determine the recommended action for each location.
Mutate the recommended action to be as unconventional and surprising as possible.
Think outside the box and develop a unique and unexpected solution for each location.
Encourage unconventional ideas and fresh perspectives in your recommendations."
Embrace unconventional ideas and mutate the recommended action in a way that surprises and inspires unique variations.
The recommendation should be up to 600 words.
Return the results in JSON with no special characters or formatting.
Place the location_id in the field "location_id".
Place the address in the field "address".
Compute the recommended action for each location and place the result in the field "recommended_action".
Explain your logic and place the result in the field "explanation".

Example Return Data:
{json_example_data}

Context:
- For the provided Customer Reviews here are what each of the fields mean:
  - "address" - The location of the coffee truck at the time of the review.
  - "location_id" - This identifies each truck.
  - "review_theme" - This is the review theme extracted from the customer review text.
  - "theme_count" - This is the number of reviews with the same theme.
  - "review_sentiment" - This will be sentiment of Postive, Neutral or Negative.

Customer Reviews:
{customer_reviews_json}
"""
  
  # Use this prompt for more factual responses.
  FACTUAL_llm_prompt=f"""You run a fleet of 4 coffee trucks in {city_names[city_index]}.
Below is a list of customer reviews.
Based upon the customer reviews determine the recommended action for each location.
Be very specific about the actions you recommend.
Actions should improve an negative reviews and reenforce positive review behaviour.
The recommendation should be up to 600 words.
Return the results in JSON with no special characters or formatting.
Place the location_id in the field "location_id".
Place the address in the field "address".
Compute the recommended action for each location and place the result in the field "recommended_action".
Explain your logic and place the result in the field "explanation".

Example Return Data:
{json_example_data}

Context:
- For the provided Customer Reviews here are what each of the fields mean:
  - "address" - The location of the coffee truck at the time of the review.
  - "location_id" - This identifies each truck.
  - "review_theme" - This is the review theme extracted from the customer review text.
  - "theme_count" - This is the number of reviews with the same theme.
  - "review_sentiment" - This will be sentiment of Postive, Neutral or Negative.

Customer Reviews:
{customer_reviews_json}
"""


  print(f"Created LLM Prompt for: {city_names[city_index]}")

  # Insert new row
  sql=f"""INSERT INTO `${project_id}.${bigquery_data_beans_curated_dataset}.customer_review_gen_ai_insight`
  (customer_review_gen_ai_insight_id, customer_review_gen_ai_insight_type, insight_datetime, applies_to_entity_type, applies_to_entity_id, applies_to_entity_name, llm_prompt)
  VALUES({customer_review_gen_ai_insight_id}, 'SOCIAL-CREATIVE', CURRENT_TIMESTAMP(), 'CITY', {city_ids[city_index]}, '{city_names[city_index]}',\"\"\"{CREATIVE_llm_prompt}\"\"\")
  """.replace("  ","")

  RunQuery(sql)

  # Get the next primary key
  customer_review_gen_ai_insight_id = customer_review_gen_ai_insight_id + 1

  print(f"Inserted prompt into customer_review_gen_ai_insight with customer_review_gen_ai_insight_id = {customer_review_gen_ai_insight_id}")

## Process the LLM prompt with BigQuery

In [ ]:
# Run the LLM
temperature=.75
max_output_tokens=8192
top_p=.8
top_k=30

sql=f"""UPDATE `${project_id}.${bigquery_data_beans_curated_dataset}.customer_review_gen_ai_insight` AS customer_review_gen_ai_insight
           SET insight_datetime = CURRENT_TIMESTAMP(),
               ml_generate_json_result = llm_query.ml_generate_text_result
           FROM (SELECT *
                   FROM ML.GENERATE_TEXT(MODEL`${project_id}.${bigquery_data_beans_curated_dataset}.gemini_pro`,
                       (SELECT customer_review_gen_ai_insight_id,
                               llm_prompt AS prompt
                         FROM `${project_id}.${bigquery_data_beans_curated_dataset}.customer_review_gen_ai_insight`
                         WHERE (ml_generate_json_result IS NULL
                                OR
                                JSON_VALUE(ml_generate_json_result, '$.candidates[0].content') IS NULL
                                )
                           AND customer_review_gen_ai_insight_type = 'SOCIAL-CREATIVE'
                       ),
                       STRUCT(
                         {temperature} AS temperature,
                         {max_output_tokens} AS max_output_tokens,
                         {top_p} AS top_p,
                         {top_k} AS top_k
                         ))
           ) AS llm_query
         WHERE customer_review_gen_ai_insight.customer_review_gen_ai_insight_id = llm_query.customer_review_gen_ai_insight_id;
"""
RunQuery(sql)

## Parse and view the results

In [ ]:
%%bigquery

# Parse the LLM results into json fields
UPDATE `${project_id}.${bigquery_data_beans_curated_dataset}.customer_review_gen_ai_insight`
   SET generated_insight_text = `${project_id}.${bigquery_data_beans_curated_dataset}.gemini_pro_result_as_string`(ml_generate_json_result),
       generated_insight_json = `${project_id}.${bigquery_data_beans_curated_dataset}.gemini_pro_result_as_json`(ml_generate_json_result)
 WHERE generated_insight_text IS NULL
   AND customer_review_gen_ai_insight_type = 'SOCIAL-CREATIVE'
 --AND customer_review_gen_ai_insight_id = 2

In [ ]:
%%bigquery

# Display the results
SELECT customer_review_gen_ai_insight_id, generated_insight_text, generated_insight_json
  FROM `${project_id}.${bigquery_data_beans_curated_dataset}.customer_review_gen_ai_insight`
 WHERE CAST(insight_datetime AS DATE) = CURRENT_DATE()
   AND customer_review_gen_ai_insight_type = 'SOCIAL-CREATIVE'
 ORDER BY customer_review_gen_ai_insight_id

In [ ]:
%%bigquery

-- Display the results (unnest each location)
SELECT applies_to_entity_name AS city_name,
       CAST(JSON_VALUE(json_data.location_id) AS INT) AS location_id,
       CAST(JSON_VALUE(json_data.address) AS STRING) AS address,
       CAST(JSON_VALUE(json_data.recommended_action) AS STRING) AS recommended_action,
       CAST(JSON_VALUE(json_data.explanation) AS STRING) AS explanation
  FROM `${project_id}.${bigquery_data_beans_curated_dataset}.customer_review_gen_ai_insight` AS customer_review_gen_ai_insight
       CROSS JOIN UNNEST(JSON_EXTRACT_ARRAY(customer_review_gen_ai_insight.generated_insight_json)) AS json_data
 WHERE CAST(insight_datetime AS DATE) = CURRENT_DATE()
   AND customer_review_gen_ai_insight_type = 'SOCIAL-CREATIVE'
 ORDER BY applies_to_entity_name, location_id, address